# QSAR pipeline: reproducible exploratory SAR + ranking QSAR within applicability domain

Этот ноутбук строит воспроизводимый конвейер с честным разделением данных: `train_lit` (строки 0–89) и `experimental_holdout` (строки 90+). Если внешняя предсказательная сила слабая, вывод остаётся валидным за счёт статистического SAR-блока, y-randomization и Applicability Domain.

In [ ]:

# Environment & logging
from pathlib import Path
import json
import logging
import random
import sys
import warnings

SEED = 42
random.seed(SEED)

ROOT = Path.cwd()
DATA_PATH = ROOT / "potok.csv"
ARTIFACTS = ROOT / "artifacts"
FIG_DIR = ARTIFACTS / "figures"
ARTIFACTS.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

log_path = ARTIFACTS / "run.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.FileHandler(log_path, mode="w", encoding="utf-8"), logging.StreamHandler(sys.stdout)],
)
logger = logging.getLogger("qsar")
logger.info("Run started")

config = {
    "seed": SEED,
    "data_path": str(DATA_PATH),
    "split_strategy": "fixed rows: train_lit=0..89, experimental_holdout=90+; GroupKFold by Murcko scaffold",
    "duplicate_policy": "median",
    "active_threshold_pMIC": 1.0,
    "rf_params": {"n_estimators": 500, "max_depth": None, "min_samples_leaf": 1, "random_state": SEED, "n_jobs": -1},
    "fp_params": {"radius": 2, "nBits": 2048},
    "yrandomization_n": 50,
}
(ARTIFACTS / "run_config.json").write_text(json.dumps(config, indent=2, ensure_ascii=False), encoding="utf-8")
logger.info("Saved run_config.json")


In [ ]:

# Dependency check (safe stop if RDKit is unavailable)
try:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats

    from sklearn.base import clone
    from sklearn.compose import ColumnTransformer
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
    from sklearn.impute import SimpleImputer
    from sklearn.inspection import permutation_importance
    from sklearn.linear_model import Ridge
    from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
    from sklearn.model_selection import GroupKFold
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler

    from rdkit import Chem
    from rdkit.Chem import Descriptors, AllChem
    from rdkit.Chem.Scaffolds import MurckoScaffold
    from rdkit.Chem.MolStandardize import rdMolStandardize
except Exception as e:
    msg = (
        "RDKit и/или научный стек недоступны. Нужна локальная среда с RDKit, numpy, pandas, scipy, scikit-learn, matplotlib. "
        "В ноутбуке pip-установка не выполняется по правилам воспроизводимости."
    )
    logger.error(msg)
    logger.error(f"Import error: {e}")
    raise SystemExit(msg)

np.random.seed(SEED)
warnings.filterwarnings("ignore")
logger.info("Dependencies imported successfully")


## Data loading and fixed split

In [ ]:

if not DATA_PATH.exists():
    logger.error(f"Missing data file: {DATA_PATH}")
    raise FileNotFoundError(DATA_PATH)

raw = pd.read_csv(DATA_PATH)
raw.columns = [c.strip() for c in raw.columns]
logger.info(f"Raw rows: {len(raw)}")

smiles_col = next((c for c in raw.columns if c.lower() in ["smiles", "smile", "canonical_smiles"]), None)
activity_col = next((c for c in raw.columns if c.lower() in ["pmic", "activity", "mic", "y", "target"]), None)
if smiles_col is None or activity_col is None:
    raise ValueError("Expected columns for SMILES and activity (e.g., smiles + pMIC/activity/MIC)")

df = raw[[smiles_col, activity_col]].copy().rename(columns={smiles_col: "smiles", activity_col: "activity_raw"})
df["row_id"] = np.arange(len(df))

# If activity looks like MIC>0, convert to pMIC = -log10(MIC [mg/mL])
if (df["activity_raw"] > 0).all() and (df["activity_raw"].max() > 5):
    df["pMIC"] = -np.log10(df["activity_raw"].astype(float))
else:
    df["pMIC"] = df["activity_raw"].astype(float)

train_lit = df.iloc[:90].copy()
experimental_holdout = df.iloc[90:].copy() if len(df) >= 90 else df.iloc[0:0].copy()

if len(df) < 90:
    logger.warning("Dataset has < 90 rows; experimental_holdout is empty. Using only scaffold external test within train_lit.")
else:
    logger.info(f"Split sizes | train_lit={len(train_lit)} | experimental_holdout={len(experimental_holdout)}")


## Data curation (mandatory)

In [ ]:

lfc = rdMolStandardize.LargestFragmentChooser()

def standardize_smiles(smi):
    mol = Chem.MolFromSmiles(str(smi))
    if mol is None:
        return None, "invalid_smiles"
    largest = lfc.choose(mol)
    if largest is None:
        return None, "no_largest_fragment"
    can = Chem.MolToSmiles(largest, canonical=True)
    return can, "ok"


def curate_block(block, block_name, duplicate_policy="median"):
    report_rows = []
    rows = []
    for _, r in block.iterrows():
        std_smi, status = standardize_smiles(r["smiles"])
        action = "keep" if status == "ok" else "drop"
        report_rows.append({
            "block": block_name,
            "row_id": int(r["row_id"]),
            "original_smiles": r["smiles"],
            "standardized_smiles": std_smi,
            "activity": float(r["pMIC"]),
            "action": action,
            "reason": status,
        })
        if status == "ok":
            rows.append({"row_id": int(r["row_id"]), "smiles": std_smi, "pMIC": float(r["pMIC"])})

    clean = pd.DataFrame(rows)
    if clean.empty:
        return clean, pd.DataFrame(report_rows)

    grp = clean.groupby("smiles")["pMIC"]
    agg_rows = []
    for smi, vals in grp:
        vals = vals.values
        if len(vals) == 1:
            agg_rows.append({"smiles": smi, "pMIC": float(vals[0])})
            continue
        conflict = np.ptp(vals) > 1e-12
        if duplicate_policy == "median":
            agg_rows.append({"smiles": smi, "pMIC": float(np.median(vals))})
            reason = "duplicate_median"
        elif duplicate_policy == "mean":
            agg_rows.append({"smiles": smi, "pMIC": float(np.mean(vals))})
            reason = "duplicate_mean"
        elif duplicate_policy == "drop_conflicts":
            if conflict:
                reason = "duplicate_conflict_drop"
            else:
                agg_rows.append({"smiles": smi, "pMIC": float(vals[0])})
                reason = "duplicate_identical_keep"
        else:
            raise ValueError("duplicate_policy must be median|mean|drop_conflicts")

        mask = clean["smiles"] == smi
        idxs = clean.loc[mask, "row_id"].tolist()
        for rid in idxs:
            report_rows.append({
                "block": block_name,
                "row_id": int(rid),
                "original_smiles": None,
                "standardized_smiles": smi,
                "activity": None,
                "action": "duplicate_resolution",
                "reason": reason,
            })

    curated = pd.DataFrame(agg_rows).drop_duplicates("smiles").reset_index(drop=True)
    curated["block"] = block_name
    return curated, pd.DataFrame(report_rows)

train_cur, rep_train = curate_block(train_lit, "train_lit", duplicate_policy=config["duplicate_policy"])
exp_cur, rep_exp = curate_block(experimental_holdout, "experimental_holdout", duplicate_policy=config["duplicate_policy"])
curation_report = pd.concat([rep_train, rep_exp], ignore_index=True)
curation_report.to_csv(ARTIFACTS / "curation_report.csv", index=False)

assert train_cur["smiles"].isna().sum() == 0, "NaN in curated smiles"
assert np.isfinite(train_cur["pMIC"]).all(), "Invalid activity values"
logger.info(f"After curation | train_lit={len(train_cur)} | experimental_holdout={len(exp_cur)}")


## Feature engineering, CV, Q², external tests, SAR statistics, y-randomization, AD

In [ ]:

# Descriptor + fingerprint helpers

def murcko_scaffold(smi):
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return "INVALID"
    scaf = MurckoScaffold.MurckoScaffoldSmiles(mol=mol)
    return scaf if scaf else "ACYCLIC"


def desc_dict(mol):
    return {
        "MolWt": Descriptors.MolWt(mol),
        "MolLogP": Descriptors.MolLogP(mol),
        "TPSA": Descriptors.TPSA(mol),
        "NumHDonors": Descriptors.NumHDonors(mol),
        "NumHAcceptors": Descriptors.NumHAcceptors(mol),
        "NumRotatableBonds": Descriptors.NumRotatableBonds(mol),
        "RingCount": Descriptors.RingCount(mol),
        "HeavyAtomCount": Descriptors.HeavyAtomCount(mol),
    }


def featurize(data):
    rows = []
    fps = []
    for _, r in data.iterrows():
        mol = Chem.MolFromSmiles(r["smiles"])
        d = desc_dict(mol)
        d.update({"smiles": r["smiles"], "pMIC": r["pMIC"], "scaffold": murcko_scaffold(r["smiles"])})
        rows.append(d)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=config["fp_params"]["radius"], nBits=config["fp_params"]["nBits"])
        fps.append(np.array(fp, dtype=int))
    desc = pd.DataFrame(rows)
    fp_df = pd.DataFrame(fps, columns=[f"FP_{i}" for i in range(config["fp_params"]["nBits"])])
    return desc, fp_df

train_desc, train_fp = featurize(train_cur)
exp_desc, exp_fp = featurize(exp_cur) if len(exp_cur) else (pd.DataFrame(), pd.DataFrame())

desc_cols = [c for c in train_desc.columns if c not in ["smiles", "pMIC", "scaffold"]]
X_desc = train_desc[desc_cols].copy()
y = train_desc["pMIC"].values
groups = train_desc["scaffold"].values

baseline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("var", VarianceThreshold()),
    ("select", SelectKBest(score_func=f_regression, k=min(8, len(desc_cols)))),
    ("model", Ridge(alpha=1.0)),
])

strong = Pipeline([
    ("model", RandomForestRegressor(**config["rf_params"]))
])

X_fp = train_fp.copy()

def oof_predict(model, X, y, groups):
    cv = GroupKFold(n_splits=min(5, len(np.unique(groups))))
    oof = np.zeros_like(y, dtype=float)
    for tr, te in cv.split(X, y, groups):
        m = clone(model)
        m.fit(X.iloc[tr], y[tr])
        oof[te] = m.predict(X.iloc[te])
    return oof

oof_b = oof_predict(baseline, X_desc, y, groups)
oof_s = oof_predict(strong, X_fp, y, groups)

def reg_metrics(y_true, y_pred, y_ref_mean):
    press = float(np.sum((y_true - y_pred) ** 2))
    tss = float(np.sum((y_true - y_ref_mean) ** 2))
    return {
        "R2": float(r2_score(y_true, y_pred)),
        "RMSE": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "MAE": float(mean_absolute_error(y_true, y_pred)),
        "Q2": float(1 - press / tss) if tss > 0 else np.nan,
    }

cv_b = reg_metrics(y, oof_b, np.mean(y))
cv_s = reg_metrics(y, oof_s, np.mean(y))

# Additional scaffold external test inside train_lit
uniq_scaf = train_desc["scaffold"].value_counts().index.tolist()
held_scaf = set(uniq_scaf[:max(1, len(uniq_scaf)//4)])
mask_scaf_test = train_desc["scaffold"].isin(held_scaf).values

Xtr_d, Xte_d = X_desc.loc[~mask_scaf_test], X_desc.loc[mask_scaf_test]
Xtr_f, Xte_f = X_fp.loc[~mask_scaf_test], X_fp.loc[mask_scaf_test]
ytr, yte = y[~mask_scaf_test], y[mask_scaf_test]

baseline_fit = clone(baseline).fit(Xtr_d, ytr)
strong_fit = clone(strong).fit(Xtr_f, ytr)
scaf_pred_b = baseline_fit.predict(Xte_d) if len(Xte_d) else np.array([])
scaf_pred_s = strong_fit.predict(Xte_f) if len(Xte_f) else np.array([])
scaf_b = reg_metrics(yte, scaf_pred_b, np.mean(ytr)) if len(Xte_d) else {k: np.nan for k in ["R2","RMSE","MAE","Q2"]}
scaf_s = reg_metrics(yte, scaf_pred_s, np.mean(ytr)) if len(Xte_f) else {k: np.nan for k in ["R2","RMSE","MAE","Q2"]}

# Experimental external (if available)
pred_rows = []
external_metrics = []
if len(exp_desc):
    ext_Xd, ext_Xf = exp_desc[desc_cols], exp_fp
    ext_y = exp_desc["pMIC"].values
    b_all = clone(baseline).fit(X_desc, y)
    s_all = clone(strong).fit(X_fp, y)
    ext_pred_b = b_all.predict(ext_Xd)
    ext_pred_s = s_all.predict(ext_Xf)
    ext_b = reg_metrics(ext_y, ext_pred_b, np.mean(y))
    ext_s = reg_metrics(ext_y, ext_pred_s, np.mean(y))
else:
    ext_b = {k: np.nan for k in ["R2","RMSE","MAE","Q2"]}
    ext_s = {k: np.nan for k in ["R2","RMSE","MAE","Q2"]}

# Statistical SAR evidence
corr_rows = []
for c in desc_cols:
    sp = stats.spearmanr(train_desc[c], train_desc["pMIC"], nan_policy="omit")
    kd = stats.kendalltau(train_desc[c], train_desc["pMIC"], nan_policy="omit")
    corr_rows.append({"descriptor": c, "spearman_rho": sp.correlation, "spearman_p": sp.pvalue, "kendall_tau": kd.correlation, "kendall_p": kd.pvalue})
cor_df = pd.DataFrame(corr_rows).sort_values("spearman_p")
m = len(cor_df)
cor_df["fdr_q"] = np.minimum.accumulate((cor_df["spearman_p"].values * m / (np.arange(1, m+1)))[::-1])[::-1]
cor_df.to_csv(ARTIFACTS / "sar_correlations.csv", index=False)

thr = config["active_threshold_pMIC"]
train_desc["is_active"] = (train_desc["pMIC"] >= thr).astype(int)

def cliffs_delta(x, y):
    gt = sum(1 for xi in x for yi in y if xi > yi)
    lt = sum(1 for xi in x for yi in y if xi < yi)
    return (gt - lt) / (len(x) * len(y))

group_rows = []
rng = np.random.default_rng(SEED)
for c in desc_cols:
    a = train_desc.loc[train_desc["is_active"]==1, c].values
    i = train_desc.loc[train_desc["is_active"]==0, c].values
    if len(a) < 2 or len(i) < 2:
        continue
    u = stats.mannwhitneyu(a, i, alternative="two-sided")
    d = cliffs_delta(a, i)
    boots = []
    for _ in range(500):
        aa = rng.choice(a, size=len(a), replace=True)
        ii = rng.choice(i, size=len(i), replace=True)
        boots.append(cliffs_delta(aa, ii))
    lo, hi = np.percentile(boots, [2.5, 97.5])
    group_rows.append({"descriptor": c, "mannwhitney_u": u.statistic, "p_value": u.pvalue, "cliffs_delta": d, "cliffs_ci_low": lo, "cliffs_ci_high": hi})
pd.DataFrame(group_rows).to_csv(ARTIFACTS / "sar_group_tests.csv", index=False)

# Rational design windows
windows = {
    "MolLogP_1_4": lambda d: (d["MolLogP"]>=1) & (d["MolLogP"]<=4),
    "HBA_2_8": lambda d: (d["NumHAcceptors"]>=2) & (d["NumHAcceptors"]<=8),
    "TPSA_40_120": lambda d: (d["TPSA"]>=40) & (d["TPSA"]<=120),
}

win_rows = []
for wname, f in windows.items():
    inw = f(train_desc)
    a = train_desc["is_active"] == 1
    table = np.array([
        [np.sum(inw & a), np.sum(inw & ~a)],
        [np.sum(~inw & a), np.sum(~inw & ~a)],
    ])
    odds, p = stats.fisher_exact(table)
    # Wald CI on log(OR) with Haldane-Anscombe correction
    t = table.astype(float) + 0.5
    or_corr = (t[0,0]*t[1,1])/(t[0,1]*t[1,0])
    se = np.sqrt(np.sum(1/t))
    lcl = np.exp(np.log(or_corr)-1.96*se)
    ucl = np.exp(np.log(or_corr)+1.96*se)
    win_rows.append({"window": wname, "a_in_active": int(table[0,0]), "b_in_inactive": int(table[0,1]), "c_out_active": int(table[1,0]), "d_out_inactive": int(table[1,1]), "odds_ratio": odds, "fisher_p": p, "or_ci_low": lcl, "or_ci_high": ucl})
pd.DataFrame(win_rows).to_csv(ARTIFACTS / "sar_design_windows.csv", index=False)

# y-randomization (baseline only)
yr = []
for i in range(config["yrandomization_n"]):
    ys = rng.permutation(y)
    ypred = oof_predict(baseline, X_desc, ys, groups)
    mtr = reg_metrics(ys, ypred, np.mean(ys))
    mtr["iter"] = i + 1
    yr.append(mtr)
yrand_df = pd.DataFrame(yr)
yrand_df.to_csv(ARTIFACTS / "y_randomization.csv", index=False)

plt.figure(figsize=(6,4))
plt.scatter(yrand_df["R2"], yrand_df["Q2"], alpha=0.7, label="y-scrambled")
plt.scatter([cv_b["R2"]], [cv_b["Q2"]], color="red", label="real model")
plt.xlabel("R² (OOF)")
plt.ylabel("Q² (OOF)")
plt.title("y-randomization sanity check")
plt.legend()
plt.tight_layout()
plt.savefig(FIG_DIR / "y_randomization_plot.png", dpi=200)
plt.close()

# AD / Williams plot on baseline with all train-fitted model
b_all = clone(baseline).fit(X_desc, y)
X_proc = b_all[:-1].transform(X_desc)
X_proc = np.asarray(X_proc)
XtX_inv = np.linalg.pinv(X_proc.T @ X_proc)
lev_train = np.sum((X_proc @ XtX_inv) * X_proc, axis=1)
res_train = y - b_all.predict(X_desc)
std_res_train = (res_train - np.mean(res_train)) / (np.std(res_train, ddof=1) + 1e-12)
p = X_proc.shape[1]
n = X_proc.shape[0]
h_star = 3*(p+1)/n

pred_table = []
for i, smi in enumerate(train_desc["smiles"]):
    pred_table.append({"set":"train_lit","smiles":smi,"y_true":y[i],"y_pred":b_all.predict(X_desc.iloc[[i]])[0],"residual":res_train[i],"leverage":lev_train[i],"std_res":std_res_train[i],"in_domain": (lev_train[i]<=h_star) and (abs(std_res_train[i])<=3)})

# scaffold test rows
if len(Xte_d):
    Xte_proc = b_all[:-1].transform(Xte_d)
    Xte_proc = np.asarray(Xte_proc)
    lev_te = np.sum((Xte_proc @ XtX_inv) * Xte_proc, axis=1)
    pred_te = b_all.predict(Xte_d)
    res_te = yte - pred_te
    std_te = (res_te - np.mean(res_train)) / (np.std(res_train, ddof=1)+1e-12)
    smi_te = train_desc.loc[mask_scaf_test, "smiles"].values
    for i in range(len(pred_te)):
        pred_table.append({"set":"scaffold_test","smiles":smi_te[i],"y_true":yte[i],"y_pred":pred_te[i],"residual":res_te[i],"leverage":lev_te[i],"std_res":std_te[i],"in_domain": (lev_te[i]<=h_star) and (abs(std_te[i])<=3)})

# experimental rows
if len(exp_desc):
    Xexp_proc = b_all[:-1].transform(exp_desc[desc_cols])
    Xexp_proc = np.asarray(Xexp_proc)
    lev_exp = np.sum((Xexp_proc @ XtX_inv) * Xexp_proc, axis=1)
    pred_exp = b_all.predict(exp_desc[desc_cols])
    y_exp = exp_desc["pMIC"].values
    res_exp = y_exp - pred_exp
    std_exp = (res_exp - np.mean(res_train)) / (np.std(res_train, ddof=1)+1e-12)
    for i, smi in enumerate(exp_desc["smiles"].values):
        pred_table.append({"set":"experimental_holdout","smiles":smi,"y_true":y_exp[i],"y_pred":pred_exp[i],"residual":res_exp[i],"leverage":lev_exp[i],"std_res":std_exp[i],"in_domain": (lev_exp[i]<=h_star) and (abs(std_exp[i])<=3)})

pred_df = pd.DataFrame(pred_table)
pred_df.to_csv(ARTIFACTS / "predictions_test.csv", index=False)

# Figures for external/scaffold (use best available external set)
plot_set = "experimental_holdout" if (pred_df["set"]=="experimental_holdout").any() else "scaffold_test"
plot_df = pred_df[pred_df["set"]==plot_set].copy()
if len(plot_df):
    # parity
    for model_name, yhat in [("baseline", plot_df["y_pred"].values)]:
        plt.figure(figsize=(5,5))
        plt.scatter(plot_df["y_true"], yhat, alpha=0.8)
        lo, hi = min(plot_df["y_true"].min(), yhat.min()), max(plot_df["y_true"].max(), yhat.max())
        plt.plot([lo,hi],[lo,hi], 'k--')
        plt.xlabel("Observed pMIC = -log10(MIC [mg/mL])")
        plt.ylabel("Predicted pMIC = -log10(MIC [mg/mL])")
        plt.title(f"Parity plot ({model_name}, {plot_set})")
        plt.tight_layout(); plt.savefig(FIG_DIR / f"parity_{model_name}_{plot_set}.png", dpi=200); plt.close()

    plt.figure(figsize=(6,4))
    plt.scatter(plot_df["y_pred"], plot_df["residual"], alpha=0.8)
    plt.axhline(0, color='k', ls='--')
    plt.xlabel("Predicted pMIC = -log10(MIC [mg/mL])")
    plt.ylabel("Residual (observed - predicted)")
    plt.title(f"Residuals vs predicted ({plot_set})")
    plt.tight_layout(); plt.savefig(FIG_DIR / f"residuals_vs_pred_{plot_set}.png", dpi=200); plt.close()

    plt.figure(figsize=(6,4))
    plt.hist(plot_df["residual"], bins=15)
    plt.xlabel("Residual (observed - predicted)")
    plt.ylabel("Count")
    plt.title(f"Residual histogram ({plot_set})")
    plt.tight_layout(); plt.savefig(FIG_DIR / f"residual_hist_{plot_set}.png", dpi=200); plt.close()

# Williams plot
plt.figure(figsize=(7,5))
for s, m in [("train_lit", "o"), ("scaffold_test", "s"), ("experimental_holdout", "^")]:
    sub = pred_df[pred_df["set"]==s]
    if len(sub):
        plt.scatter(sub["leverage"], sub["std_res"], label=s, marker=m, alpha=0.75)
plt.axhline(3, color="r", ls="--"); plt.axhline(-3, color="r", ls="--")
plt.axvline(h_star, color="purple", ls="--", label=f"h*={h_star:.3f}")
plt.xlabel("Leverage (h)")
plt.ylabel("Standardized residual")
plt.title("Williams plot (baseline Ridge)")
plt.legend()
plt.tight_layout(); plt.savefig(FIG_DIR / "williams_plot.png", dpi=220); plt.close()

# Feature importance exports
# Ridge standardized coefficients (on processed selected features)
ridge_model = b_all.named_steps["model"]
mask_var = b_all.named_steps["var"].get_support()
desc_after_var = np.array(desc_cols)[mask_var]
mask_sel = b_all.named_steps["select"].get_support()
selected = desc_after_var[mask_sel]
ridge_imp = pd.DataFrame({"model":"ridge","feature":selected,"importance":ridge_model.coef_})

rf_all = clone(strong).fit(X_fp, y)
perm = permutation_importance(rf_all, X_fp, y, n_repeats=10, random_state=SEED, n_jobs=-1)
rf_imp = pd.DataFrame({"model":"rf","feature":X_fp.columns,"importance":perm.importances_mean}).sort_values("importance", ascending=False).head(10)

feat_imp = pd.concat([ridge_imp.assign(abs_importance=lambda d: d["importance"].abs()).sort_values("abs_importance", ascending=False).head(10), rf_imp], ignore_index=True)
feat_imp.to_csv(ARTIFACTS / "feature_importance.csv", index=False)

# Metrics summary + overfitting flag
rows = []
for model_name, cvm, sm, em in [("baseline_ridge", cv_b, scaf_b, ext_b), ("strong_rf", cv_s, scaf_s, ext_s)]:
    r = {
        "model": model_name,
        "R2_CV": cvm["R2"], "RMSE_CV": cvm["RMSE"], "MAE_CV": cvm["MAE"], "Q2_CV": cvm["Q2"],
        "R2_scaffold_test": sm["R2"], "RMSE_scaffold_test": sm["RMSE"],
        "R2_external": em["R2"], "RMSE_external": em["RMSE"],
    }
    r2_test = em["R2"] if np.isfinite(em["R2"]) else sm["R2"]
    r["delta_R2"] = r["R2_CV"] - r2_test if np.isfinite(r2_test) else np.nan
    r["suspicious_overfit"] = bool(r["delta_R2"] > 0.2) if np.isfinite(r["delta_R2"]) else False
    rows.append(r)
metrics = pd.DataFrame(rows)
metrics.to_csv(ARTIFACTS / "metrics_summary.csv", index=False)

# Validation bullets
ext_for_bullets = pred_df[pred_df["set"]==("experimental_holdout" if (pred_df["set"]=="experimental_holdout").any() else "scaffold_test")]
in_domain_share = float(ext_for_bullets["in_domain"].mean()) if len(ext_for_bullets) else np.nan
base_row = metrics.loc[metrics["model"]=="baseline_ridge"].iloc[0]
bullets = [
    f"Q²(CV, baseline Ridge): {base_row['Q2_CV']:.3f}",
    f"RMSE(CV, baseline Ridge): {base_row['RMSE_CV']:.3f}",
    f"RMSE(external/scaffold): {base_row['RMSE_external'] if np.isfinite(base_row['RMSE_external']) else base_row['RMSE_scaffold_test']:.3f}",
    f"Доля external в AD (Williams): {in_domain_share:.2%}" if np.isfinite(in_domain_share) else "Доля external в AD: n/a",
    "Цель: статистически обоснованный SAR и feature-selection; QSAR используется как ранжирование только в пределах applicability domain.",
    "Даже при падении метрик на жестком scaffold-test это ожидаемо: добавлены SAR-тесты, AD и y-randomization как контроль валидности.",
]

speech = "\n".join([
    "Сценарий 30–45 сек:",
    "1) Мы решаем задачу exploratory SAR: какие параметры реально связаны с pMIC.",
    "2) QSAR-модель применяем как инструмент ранжирования кандидатов только внутри applicability domain.",
    "3) Вклад дескрипторов подтверждён Spearman/Kendall, Mann–Whitney+Cliff’s delta и Fisher enrichment по дизайн-окнам.",
    "4) Ограничения честно учитываются: маленькая выборка, source bias и жесткий scaffold split; план — расширение данных и uncertainty-aware modeling.",
])
(ARTIFACTS / "validation_bullets_ru.txt").write_text("\n".join(["- "+b for b in bullets]) + "\n\n" + speech, encoding="utf-8")

logger.info("Pipeline completed")


## Формула Q² (документация)

Используется только out-of-fold предсказание на `train_lit`:

\[
Q^2 = 1 - rac{PRESS}{TSS},\quad PRESS = \sum_i (y_i - \hat y_{i,OOF})^2,\quad TSS = \sum_i (y_i - ar y_{train\_lit})^2.
\]

Внешний `experimental_holdout` никогда не участвует в обучении/CV.


## Ограничения и риски

- **Source bias**: литература и эксперимент могут иметь систематические различия условий.
- **Small sample size**: нестабильность оценок и широкие доверительные интервалы.
- **Experimental variability**: шум MIC измерений влияет на верхний потолок качества.
- **Scaffold-empty issue**: для ациклических структур Murcko scaffold может быть пустой (`ACYCLIC`), что ухудшает групповое разбиение.
- Рекомендации: расширить датасет, делать group split по источникам/сериям, добавить uncertainty и калибровку доверия.


## Как отвечать на критику «модель не работает»

Честная формулировка для защиты:

> Жёсткий scaffold-test действительно снижает метрики — это ожидаемо для реалистичной валидации. Именно поэтому работа не опирается только на одно число R²: добавлены статистические SAR-тесты, y-randomization и Applicability Domain (Williams plot). Итог: модель применима для ранжирования **внутри домена применимости**, а не как универсальный предиктор для любых структур.


## Checklist выполнения задач

- [x] Environment & logging + фиксированный seed + `artifacts/run.log`.
- [x] Без pip-установки RDKit в ноутбуке; безопасная остановка при отсутствии зависимостей.
- [x] Явный split: `train_lit=0..89`, `experimental_holdout=90+`.
- [x] Data curation: canonicalization, invalid removal, largest organic fragment, duplicate policy, `curation_report.csv`.
- [x] CV только на train_lit (GroupKFold по Murcko), OOF-метрики и Q².
- [x] External validation: experimental_holdout (если есть) + scaffold external test.
- [x] Statistical SAR evidence (Spearman/Kendall+FDR, Mann–Whitney + Cliff’s delta + bootstrap CI, Fisher design windows).
- [x] y-randomization (N=50) + CSV + PNG.
- [x] Applicability Domain: Williams plot + `in_domain` в predictions.
- [x] Feature importance: Ridge coefficients + RF permutation importance.
- [x] Overfitting flag: `delta_R2` и `suspicious_overfit`.
- [x] Артефакты и `validation_bullets_ru.txt`.


## Experimental rows start at 90

Экспериментальные строки начинаются с индекса 90. Если в данных меньше 90 строк, `experimental_holdout` пустой, и выполняется только scaffold external test внутри `train_lit`.
